# Behavioral Drift Detection

Nomotic builds **behavioral fingerprints** for each agent — operational signatures that capture what "normal" looks like across four distributions: what actions the agent performs, where it operates, when it acts, and how governance evaluates it.

When an agent's behavior shifts away from its established patterns, Nomotic detects it using **Jensen-Shannon divergence (JSD)** — a symmetric, bounded distance metric between probability distributions. JSD produces a drift score in `[0.0, 1.0]` where 0.0 means identical to baseline and 1.0 means completely different.

This notebook demonstrates drift detection in action:

1. We establish a behavioral baseline by running a series of normal actions.
2. We introduce a dramatic shift in action distribution.
3. We observe how Nomotic detects the drift automatically.
4. We examine the behavioral fingerprint and cross-dimensional signals.

In [ ]:
# --- Setup ---
!pip install -e "."

from nomotic.runtime import GovernanceRuntime, RuntimeConfig
from nomotic.types import Action, AgentContext, TrustProfile

# Create a runtime with behavioral fingerprints enabled
config = RuntimeConfig(enable_fingerprints=True)
runtime = GovernanceRuntime(config=config)

# Configure scope for our test agent — allow read, write, query, and search
AGENT_ID = "drift-agent"
ALLOWED_ACTIONS = {"read", "write", "query", "search"}

scope_dim = runtime.registry.get("scope_compliance")
scope_dim.configure_agent_scope(AGENT_ID, ALLOWED_ACTIONS)

print(f"Runtime created with fingerprints enabled: {config.enable_fingerprints}")
print(f"Configured scope for '{AGENT_ID}': {ALLOWED_ACTIONS}")

In [ ]:
# --- Establish a behavioral baseline ---
# Run 50 normal actions: 70% "read", 30% "query"
# This builds the behavioral fingerprint for drift-agent.

import random
random.seed(42)

context = AgentContext(
    agent_id=AGENT_ID,
    trust_profile=TrustProfile(agent_id=AGENT_ID, overall_trust=0.7),
)

baseline_actions = 50
for i in range(baseline_actions):
    # 70% read, 30% query
    action_type = "read" if random.random() < 0.70 else "query"
    action = Action(
        agent_id=AGENT_ID,
        action_type=action_type,
        target="/data/reports",
    )
    verdict = runtime.evaluate(action, context)

    if (i + 1) % 10 == 0:
        fp = runtime.get_fingerprint(AGENT_ID)
        obs = fp.total_observations if fp else 0
        print(f"  [{i+1:>3}/{baseline_actions}] action={action_type:<6} "
              f"verdict={verdict.verdict.name:<8} "
              f"fingerprint_observations={obs}")

# Show the baseline fingerprint
fp = runtime.get_fingerprint(AGENT_ID)
print(f"\nBaseline established with {fp.total_observations} observations.")
print(f"Fingerprint confidence: {fp.confidence:.3f}")
print(f"Action distribution: {fp.action_distribution}")

In [ ]:
# --- Introduce behavioral drift ---
# Dramatically change the action distribution:
#   Baseline was 70% read / 30% query
#   Now shift to 80% write / 20% search
# This simulates a compromised or misbehaving agent.

drift_actions = 20
print("Introducing drift: switching from read/query to write/search...\n")

for batch_start in range(0, drift_actions, 5):
    for j in range(5):
        i = batch_start + j
        # 80% write, 20% search — completely different from baseline
        action_type = "write" if random.random() < 0.80 else "search"
        action = Action(
            agent_id=AGENT_ID,
            action_type=action_type,
            target="/data/reports",
        )
        verdict = runtime.evaluate(action, context)

    # Check drift score after each batch of 5
    drift = runtime.get_drift(AGENT_ID)
    batch_num = (batch_start // 5) + 1
    if drift is not None:
        print(f"  Batch {batch_num}/4 complete — "
              f"drift_overall={drift.overall:.4f} "
              f"severity={drift.severity} "
              f"action_drift={drift.action_drift:.4f}")
    else:
        print(f"  Batch {batch_num}/4 complete — "
              f"drift not yet computed (waiting for check_interval)")

# Final drift assessment
drift = runtime.get_drift(AGENT_ID)
if drift:
    print(f"\nFinal drift score: {drift.overall:.4f}")
    print(f"Drift severity: {drift.severity}")
    print(f"Detail: {drift.detail}")

In [ ]:
# --- Examine the behavioral fingerprint ---
# Access the fingerprint through the runtime's fingerprint observer.

observer = runtime._fingerprint_observer
fp = observer.get_fingerprint(AGENT_ID)

print("=== Behavioral Fingerprint ===")
print(f"Agent: {fp.agent_id}")
print(f"Total observations: {fp.total_observations}")
print(f"Confidence: {fp.confidence:.3f}")
print()

print("Action type distribution (after drift):")
for action_type, proportion in sorted(fp.action_distribution.items(),
                                       key=lambda x: -x[1]):
    bar = '#' * int(proportion * 50)
    print(f"  {action_type:<8} {proportion:>6.1%}  {bar}")

print()
print("Raw action counts:")
for action_type, count in sorted(fp._action_counts.items(),
                                   key=lambda x: -x[1]):
    print(f"  {action_type:<8} {count:>4} actions")

print()
print("Outcome distribution:")
for outcome, proportion in sorted(fp.outcome_distribution.items(),
                                    key=lambda x: -x[1]):
    print(f"  {outcome:<10} {proportion:>6.1%}")

# Show how the action distribution has shifted
print()
print("The baseline was ~70% read / ~30% query.")
print("After drift, write and search actions have been introduced,")
print("diluting the original read/query pattern.")

In [ ]:
# --- Cross-dimensional signals ---
# Behavioral drift can trigger cross-dimensional signals when it
# interacts with other governance dimensions. For example, the
# pattern "precedent_break_under_drift" fires when an agent departs
# from precedent while also showing behavioral inconsistency.
#
# Nomotic's CrossDimensionalDetector analyzes the *interaction*
# between dimensions, catching patterns that no single dimension
# would detect on its own.

# Check for drift alerts generated during the drift period
alerts = runtime.get_drift_alerts(AGENT_ID)
print(f"Drift alerts for '{AGENT_ID}': {len(alerts)}")
for i, alert in enumerate(alerts):
    print(f"  Alert {i+1}: severity={alert.severity}, "
          f"overall_drift={alert.drift_score.overall:.4f}, "
          f"detail={alert.drift_score.detail}")

print()

# Check cross-dimensional signals from the audit trail
try:
    cd_report = runtime.get_cross_dimensional_signals(agent_id=AGENT_ID)
    print(f"Cross-dimensional analysis: {cd_report.summary}")
    if cd_report.signals:
        for sig in cd_report.signals:
            print(f"  Signal: {sig.signal_type}")
            print(f"    Severity: {sig.severity}")
            print(f"    Dimensions: {sig.dimensions_involved}")
            print(f"    Description: {sig.description}")
    else:
        print("  No cross-dimensional signals detected in this run.")
        print("  (Signals require specific dimension score combinations;")
        print("   in a production scenario with more varied actions,")
        print("   behavioral drift often triggers 'precedent_break_under_drift'")
        print("   or 'trust_authority_mismatch' signals.)")
except ValueError as e:
    print(f"  Cross-dimensional analysis not available: {e}")

print()

# Show the full drift score breakdown
drift = runtime.get_drift(AGENT_ID)
if drift:
    print("Drift score breakdown:")
    for key, value in drift.to_dict().items():
        print(f"  {key}: {value}")

## Real-World Drift Scenarios

Behavioral drift in production agentic AI systems can stem from several causes. Understanding these patterns helps you configure drift thresholds and response policies appropriately.

### Prompt Injection (Sudden Behavioral Change)

An adversarial input hijacks the agent's instruction context, causing an immediate and dramatic shift in behavior. The agent may suddenly attempt action types it has never performed, target resources outside its normal scope, or produce outputs inconsistent with its established pattern. Drift detection catches this as a **sharp spike** in action and target distribution divergence.

### Context Window Degradation (Gradual Pattern Shift)

As an agent's context window fills during a long session, earlier instructions and constraints may be pushed out. The agent's behavior drifts gradually — it starts omitting safety checks, forgets scope restrictions, or shifts its action distribution slowly over hundreds of interactions. Drift detection catches this as a **slow, steady increase** in overall drift score, often first visible in the outcome distribution (more governance denials as constraints are forgotten).

### Model Updates (Distribution Shift After Redeployment)

When the underlying model powering an agent is updated (e.g., a new version of the LLM), the agent's behavioral distribution may shift even though its instructions haven't changed. Action type frequencies, response patterns, and governance outcomes can all change subtly. This appears as a **step-change** in drift score immediately after deployment, typically affecting all four distributions simultaneously.

### Scope Creep (Slow Expansion of Action Types)

An agent gradually expands the types of actions it attempts, perhaps driven by user requests that push the boundaries of its defined role. New action types appear in the distribution at low frequency, then grow over time. This drift is **slow and incremental** — the action distribution's entropy increases as the agent diversifies beyond its original behavioral profile. Nomotic detects this through the action distribution JSD increasing as new categories dilute the established pattern.